In [1]:
%pip install ipfn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import ipfn


## Loading the data

In [3]:
def get_data(dataset_id, sample=None):
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=" + dataset_id
    params = {'offset': 0}
    dfs = []

    while True:
        response = requests.get(url, params=params).json()
        df = pd.DataFrame(response['result']['records'])
        dfs.append(df)
        if response['result']['_links']['next'] is None or (sample is not None and len(dfs) * 100 >= sample):
            break
        params['offset'] += 100  # assuming 100 records per page

    full_df = pd.concat(dfs, ignore_index=True)
    if sample is not None:
        full_df = full_df.head(sample)
    full_df = full_df.drop(['_id'], axis=1)
    return full_df

In [4]:
#age with unemployment duration
dataset_id_1 = "d_db95e15ceffaa368a043310479dc7d57"
data_1 = get_data(dataset_id_1, 2000)

#highest education with unemployment duration
dataset_id_2 = "d_a0ca632fd1d6ff841f0e47298a9ab589"
data_2 = get_data(dataset_id_2, 2000)

#median duration of unemployment
dataset_id_3 = "d_c01a3210fb10f1a52676f97498d4ec2c"
data_3 = get_data(dataset_id_3, 2000)




In [5]:
data_1.head()

,year,sex,age,duration,unemployed
0,2010,male,15-24,under 5,4700
1,2010,male,15-24,5 to 9,1700
2,2010,male,15-24,10 to 14,700
3,2010,male,15-24,15 to 19,200
4,2010,male,15-24,20 to 24,500


In [6]:
data_2.head()

,year,sex,highest_qualification,duration,unemployed
0,2010,male,primary and below,under 5,2600
1,2010,male,primary and below,5 to 9,900
2,2010,male,primary and below,10 to 14,900
3,2010,male,primary and below,15 to 19,500
4,2010,male,primary and below,20 to 24,1300


In [7]:
data_3.head()

,year,median_dur_of_unemp
0,1991,8
1,1992,6
2,1993,6
3,1994,4
4,1996,4


In [8]:
print(data_1["duration"].unique())


['under 5' '5 to 9' '10 to 14' '15 to 19' '20 to 24' '25 to 29' '30 to 39'
 '40 to 51' '52 and over']


## Merging data

In [9]:
data_1[['unemployed']] = data_1[['unemployed']].apply(pd.to_numeric, errors='coerce')
data_2[['unemployed']] = data_2[['unemployed']].apply(pd.to_numeric, errors='coerce')
merged_data = pd.merge(data_1, data_2, on=['year', 'duration'])
merged_data = pd.merge(merged_data, data_3, on='year', how='inner')

## Preprocessing the data

In [10]:
## getting duration midpoint
def create_duration_midpoints(df):
    midpoints = {}
    for category in df["duration"].unique():
        if category == "under 5":
            midpoint = 2.5
        elif category == "52 and over":
            midpoint = 52 + (104 - 52) / 2  # assuming the upper limit is 104
        else:
            lower, upper = map(int, category.split(" to "))
            midpoint = (lower + upper) / 2
        midpoints[category] = midpoint
    return midpoints




In [11]:
midpoint_dict = create_duration_midpoints(merged_data)
merged_data['duration_midpoint'] = merged_data['duration'].map(midpoint_dict)
merged_data['censored'] = merged_data['duration'].map(lambda x: 1 if x == "52 and over" else 0)
merged_data.head()

,year,sex_x,age,duration,unemployed_x,sex_y,highest_qualification,unemployed_y,median_dur_of_unemp,duration_midpoint,censored
0,2010,male,15-24,under 5,4700.0,male,primary and below,2600.0,8,2.5,0
1,2010,male,15-24,under 5,4700.0,male,lower secondary,2200.0,8,2.5,0
2,2010,male,15-24,under 5,4700.0,male,secondary,2600.0,8,2.5,0
3,2010,male,15-24,under 5,4700.0,male,post-secondary (non-tertiary),3000.0,8,2.5,0
4,2010,male,15-24,under 5,4700.0,male,diploma and professional qualification,3200.0,8,2.5,0


In [12]:
# Marginal 1: age × sex (sum unemployed over duration)
age_sex_year_marginal = (
    data_1.groupby(['year', 'age', 'sex'])['unemployed'].sum().reset_index()
)

# Marginal 2: qualification × sex (sum unemployed over duration)
qual_sex_year_marginal = (
    data_2.groupby(['year', 'highest_qualification', 'sex'])['unemployed'].sum().reset_index()
)


In [13]:
import numpy as np
joint_tables = []

for year in data_1['year'].unique():
    # Filter marginals for the current year
    age_marginal = age_sex_year_marginal[age_sex_year_marginal['year'] == year]
    qual_marginal = qual_sex_year_marginal[qual_sex_year_marginal['year'] == year]
    
    # Pivot to get arrays
    age_pivot = age_marginal.pivot(index='age', columns='sex', values='unemployed')
    age_pivot = age_pivot.fillna(0)

    age_marg = age_pivot.values
    age_labels = age_pivot.index.tolist()
    
    qual_pivot = qual_marginal.pivot(index='highest_qualification', columns='sex', values='unemployed')
    qual_pivot = qual_pivot.fillna(0)
    
    qual_marg = qual_pivot.values
    qualification_labels = qual_pivot.index.tolist()

    sex_labels = qual_pivot.columns.tolist()
    # Initial guess
    initial_guess = np.ones((qual_marg.shape[0], age_marg.shape[0], qual_marg.shape[1]))
    
    # IPF setup
    aggregates = [qual_marg, age_marg]
    dimensions = [[0, 2], [1, 2]]  # Qualification-sex and age-sex
    
    # Run IPF
    ipf_fitter = ipfn.ipfn.ipfn(initial_guess, aggregates, dimensions)
    fitted_joint = ipf_fitter.iteration()
    print(fitted_joint)
    
    joint_tables.append((year, fitted_joint))


ipfn converged: convergence_rate not updating or below rate_tolerance
[[[3236.31713555 1777.87610619]
  [1674.93606138 1452.65486726]
  [2214.32225064 1929.6460177 ]
  [2185.93350384 1907.96460177]
  [1816.8797954  2731.85840708]]

 [[2011.76470588 1306.19469027]
  [1041.17647059 1067.25663717]
  [1376.47058824 1417.69911504]
  [1358.82352941 1401.7699115 ]
  [1129.41176471 2007.07964602]]

 [[ 874.68030691  997.78761062]
  [ 452.68542199  815.26548673]
  [ 598.46547315 1082.96460177]
  [ 590.79283887 1070.79646018]
  [ 491.04859335 1533.18584071]]

 [[1486.95652174 1179.20353982]
  [ 769.56521739  963.49557522]
  [1017.39130435 1279.86725664]
  [1004.34782609 1265.48672566]
  [ 834.7826087  1811.94690265]]

 [[1282.86445013 1396.90265487]
  [ 663.93861893 1141.37168142]
  [ 877.74936061 1516.15044248]
  [ 866.49616368 1499.11504425]
  [ 720.20460358 2146.46017699]]

 [[2507.4168798  1542.03539823]
  [1297.69820972 1259.95575221]
  [1715.60102302 1673.67256637]
  [1693.60613811 1654.86

In [14]:
# From data_1 (age-based)
duration_counts_age = data_1.groupby(['year', 'age', 'sex', 'duration'])['unemployed'].sum().reset_index()
total_age = duration_counts_age.groupby(['year', 'age', 'sex'])['unemployed'].sum().reset_index().rename(columns={'unemployed': 'total_unemployed'})
duration_probs_age = duration_counts_age.merge(total_age, on=['year', 'age', 'sex'])
duration_probs_age['probability_age'] = duration_probs_age['unemployed'] / duration_probs_age['total_unemployed']
duration_probs_age = duration_probs_age.rename(columns={'unemployed': 'unemployed_age'})
duration_probs_age = duration_probs_age[['year', 'age', 'sex', 'duration', 'unemployed_age', 'probability_age']]

# From data_2 (qualification-based)
duration_counts_qual = data_2.groupby(['year', 'highest_qualification', 'sex', 'duration'])['unemployed'].sum().reset_index()
total_qual = duration_counts_qual.groupby(['year', 'highest_qualification', 'sex'])['unemployed'].sum().reset_index().rename(columns={'unemployed': 'total_unemployed'})
duration_probs_qual = duration_counts_qual.merge(total_qual, on=['year', 'highest_qualification', 'sex'])
duration_probs_qual['probability_qual'] = duration_probs_qual['unemployed'] / duration_probs_qual['total_unemployed']
duration_probs_qual = duration_probs_qual.rename(columns={'unemployed': 'unemployed_qual'})
duration_probs_qual = duration_probs_qual[['year', 'highest_qualification', 'sex', 'duration', 'unemployed_qual', 'probability_qual']]



# Merge age-based and qualification-based probabilities
hybrid_probs = duration_probs_age.merge(
    duration_probs_qual,
    left_on=['year', 'sex', 'duration'],
    right_on=['year', 'sex', 'duration'],
    how='outer'
)


# Fill missing values with zero (if any combination is missing in one source)
hybrid_probs['probability_age'] = hybrid_probs['probability_age'].fillna(0)
hybrid_probs['probability_qual'] = hybrid_probs['probability_qual'].fillna(0)

# Set weight (e.g., 0.5 for equal weight)
w = 0.3

# Compute hybrid probability
hybrid_probs['hybrid_probability'] = (
    w * hybrid_probs['probability_age'] + (1 - w) * hybrid_probs['probability_qual']
)
print(hybrid_probs)


      year          age     sex  duration  unemployed_age  probability_age  \
0     2010        15-24  female  10 to 14           600.0         0.052632   
1     2010        15-24  female  10 to 14           600.0         0.052632   
2     2010        15-24  female  10 to 14           600.0         0.052632   
3     2010        15-24  female  10 to 14           600.0         0.052632   
4     2010        15-24  female  10 to 14           600.0         0.052632   
...    ...          ...     ...       ...             ...              ...   
7555  2023  50 and over    male   under 5          4400.0         0.289474   
7556  2023  50 and over    male   under 5          4400.0         0.289474   
7557  2023  50 and over    male   under 5          4400.0         0.289474   
7558  2023  50 and over    male   under 5          4400.0         0.289474   
7559  2023  50 and over    male   under 5          4400.0         0.289474   

                       highest_qualification  unemployed_qual  

In [16]:
joint_results = []

for i, (year, fitted_joint) in enumerate(joint_tables):
    
    for q_idx, qual in enumerate(qualification_labels):
        for a_idx, age in enumerate(age_labels):
            for s_idx, sex in enumerate(sex_labels):
                joint_results.append({
                    'year': year,
                    'highest_qualification': qual,
                    'age': age,
                    'sex': sex,
                    'joint_count': fitted_joint[q_idx, a_idx, s_idx]
                })

joint_df = pd.DataFrame(joint_results)

# Merge with hybrid probabilities and ensure no missing values
merged_df = joint_df.merge(hybrid_probs, on=['year', 'age', 'sex', 'highest_qualification'], how='left')

# Check for NaN values in hybrid_probability that would cause issues
missing_probs = merged_df['hybrid_probability'].isna().sum()
if missing_probs > 0:
    print(f"Warning: {missing_probs} rows have missing probability values")
    # Could impute or handle these cases

# Check that probabilities sum to 1 for each group
prob_sums = merged_df.groupby(['year', 'age', 'sex', 'highest_qualification'])['hybrid_probability'].sum()
if not np.allclose(prob_sums, 1.0, rtol=0.01):
    print("Warning: Probabilities don't sum to 1 for some groups")
    # Could normalize probabilities

# Allocate unemployed counts across duration
merged_df['estimated_unemployed'] = merged_df['joint_count'] * merged_df['hybrid_probability']
merged_df.tail(100)

,year,highest_qualification,age,sex,joint_count,duration,unemployed_age,probability_age,unemployed_qual,probability_qual,hybrid_probability,estimated_unemployed
7460,2023,primary and below,50 and over,female,556.103286,under 5,3800.0,0.368932,1200.0,0.521739,0.475897,264.647887
7461,2023,primary and below,50 and over,male,1104.116223,10 to 14,1900.0,0.125000,400.0,0.133333,0.130833,144.455206
7462,2023,primary and below,50 and over,male,1104.116223,15 to 19,800.0,0.052632,0.0,0.000000,0.015789,17.433414
7463,2023,primary and below,50 and over,male,1104.116223,20 to 24,1000.0,0.065789,300.0,0.100000,0.089737,99.079903
7464,2023,primary and below,50 and over,male,1104.116223,25 to 29,200.0,0.013158,0.0,0.000000,0.003947,4.358354
...,...,...,...,...,...,...,...,...,...,...,...,...
7555,2023,secondary,50 and over,male,2061.016949,30 to 39,600.0,0.039474,100.0,0.017857,0.024342,50.169492
7556,2023,secondary,50 and over,male,2061.016949,40 to 51,600.0,0.039474,100.0,0.017857,0.024342,50.169492
7557,2023,secondary,50 and over,male,2061.016949,5 to 9,2500.0,0.164474,1000.0,0.178571,0.174342,359.322034
7558,2023,secondary,50 and over,male,2061.016949,52 and over,3200.0,0.210526,300.0,0.053571,0.100658,207.457627


In [17]:
merged_df.shape

(7560, 12)

In [18]:
merged_df.to_csv("datasets/unemployment_survival_data.csv", index=False)